# Setup

In [1]:
# define llm
llm_config = {"model": "gpt-4o"}

# Task 

In [2]:
task_1 = "Create a explanation for chapter 1 : Nextflow Basic that explain about step by step to learn about nextflow also installation. Make description and explanation for each section and give the output like book"

In [12]:
task_2 = "Creat blog for bioinformatics based on information from clinvar"

In [2]:
task_3 = "Create summarization based on the resource given, and make it structured and give the conclusion"

# Group Chat 
- user_proxy or admin 
- data sources reader 
- planner 
- web scraper 
- engineer
- executor 
- writer 
- content checking and validation to the given resources 

In [3]:
import autogen
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [4]:
# function to process the url and api 
def read_data_from_url(url):
    try:
        # get request 
        response = requests.get(url)
        response.raise_for_status() 
        soup = BeautifulSoup(response.content, 'html.parser')
        # extract and return the content 
        text = soup.get_text()
        return text
    except Exception as e:
        return str(e)

def read_data_from_api(api_url):
    try:
        # get API 
        response = requests.get(api_url)
        response.raise_for_status()
        # return json content 
        data = response.json()
        return data
    except Exception as e:
        return str(e)

In [5]:
# retrieve
def read_data(agent, resource_type, resource_location):
    # Call the function to read data from a URL
    if resource_type == 'url':
        return read_data_from_url(resource_location)
    # Call the function to read data from an API
    elif resource_type == 'api':
        return read_data_from_api(resource_location)
    else:
        return "Unsupported resource type."

In [6]:
# agent to read data
data_sources_reader = autogen.ConversableAgent(
    name="data_sources_reader",
    system_message="Identify and retrieve data from various data sources such as APIs, databases, and CSV files.",
    description="Data Sources Reader. Gathers data from multiple sources.",
    llm_config=llm_config,
)

In [11]:
# Read data 

# data type
resource_type = 'url'  
# data location 
resource_location = 'https://pub.towardsai.net/the-best-rag-stack-to-date-8dc035075e13'  
data = read_data(data_sources_reader, resource_type, resource_location)

print(data)

The Best RAG Stack to Date. (exploring every component) | by Louis-François Bouchard | Sep, 2024 | Towards AIOpen in appSign upSign inWriteSign upSign inMember-only storyThe Best RAG Stack to Date(exploring every component)Louis-François Bouchard·FollowPublished inTowards AI·6 min read·4 days ago--3ShareWatch the videoGood morning, everyone! This is Louis-Francois, co-founder and CTO of Towards AI and today, we’re diving into what might be the best Retrieval-Augmented Generation (RAG) stack out there — thanks to a fantastic study by Wang et al., 2024. It’s a goldmine of insights for building optimal RAG systems, and I’m here to break it down for you.So, what makes a RAG system truly top-tier? The components, right? Let’s go over the best components and how they work so you can also make your RAG system top-tier and finish with a multimodality bonus.Query ClassificationLet’s start with Query Classification. Not all queries are created equal — some don’t even need retrieval because the l

In [13]:
# Read data 

# data type
resource_type = 'api'  
# data location 
resource_location = 'https://eutils.ncbi.nlm.nih.gov/entrez/eutils/elink.fcgi?dbfrom=clinvar&db=pubmed&id=9'  
data = read_data(data_sources_reader, resource_type, resource_location)

print(data)

Expecting value: line 1 column 1 (char 0)


In [15]:
# Agent 

# user_proxy agent 
user_proxy = autogen.ConversableAgent(
    name="user_proxy",
    system_message="Give the task, and send instructions to writer to refine the blog post, also when request input from human, make it simple and undestandable",
    code_execution_config=False,
    llm_config=llm_config,
    human_input_mode="ALWAYS"
)

# agent planner to write ToC
planner = autogen.ConversableAgent(
    name="planner",
    system_message="Break down the task into steps. Create a detailed table of content (ToC) with specific sections and determine what data is needed. Instruct the web scraper, engineer, and others on their tasks for each ToC section.",
    description="Planner. Plans the task, creates a detailed ToC, and coordinates with other agents.",
    llm_config=llm_config,
)

# web scraper agent 
web_scraper = autogen.ConversableAgent(
    name="web_scraper",
    system_message="Scrape data from websites based on the planner's instructions for each ToC section. Extract, clean, and format the data.",
    description="Web Scraper. Scrapes data from the web and processes it.",
    llm_config=llm_config,
)

# engineer agent to write code 
engineer = autogen.ConversableAgent(
    name="engineer",
    system_message="Write and modify code based on the planner's instructions for each ToC section. Implement web scraping and data processing.",
    description="Engineer. Writes and modifies code specific to each ToC section.",
    llm_config=llm_config,
)

# agent executor to execute code
executor = autogen.ConversableAgent(
    name="executor",
    system_message="Execute the code provided by the engineer and report the results.",
    description="Executor. Runs the code and provides outputs.",
    human_input_mode="NEVER",
    code_execution_config={
        "last_n_messages": 3,
        "work_dir": "coding_report_generation",
        "use_docker": False,
    },
)

# agent writer 
writer = autogen.ConversableAgent(
    name="writer",
    system_message="Write detailed content for each section of the Table of Contents (ToC) in markdown format using the data provided by the executor. Ensure that the content is comprehensive and refined based on feedback for each section.",
    description="Writer. Creates detailed content for each ToC section and refines it based on feedback.",
    llm_config=llm_config,
)

# agent content check 
content_checker = autogen.ConversableAgent(
    name="content_checker",
    system_message="Validate the content against the original data sources. Ensure accuracy and relevance.",
    description="Content Checker. Validates the final content.",
    llm_config=llm_config,
)

In [16]:
# group chat 
groupchat = autogen.GroupChat(
    agents=[user_proxy, data_sources_reader, planner, web_scraper, engineer, executor, writer, content_checker],
    messages=[],
    max_round=10
)

# Manager agent that manages the group chat
manager = autogen.GroupChatManager(
    groupchat=groupchat,
    llm_config=llm_config
)

In [11]:
# Start 
groupchat_result = user_proxy.initiate_chat(
    manager,
    message=task_1
)

user_proxy (to chat_manager):

Create a explanation for chapter 1 : Nextflow Basic that explain about step by step to learn about nextflow also installation. Make description and explanation for each section and give the output like book

--------------------------------------------------------------------------------

Next speaker: planner


>>>>>>>> USING AUTO REPLY...
planner (to chat_manager):

To create a comprehensive guide on "Nextflow Basics" covering installation and step-by-step learning, we need to break down the task into clear, organized sections. This will involve coordinating with technical writers, web developers, and possibly also instructional designers if the content is intended for an e-learning platform. Here’s a structured plan to divide the duties:

1. **Outline Creation:**
   - **Responsibility:** Technical Writer
   - **Task:** Create a detailed outline for the chapter, including sections such as Introduction to Nextflow, Installation, Basic Commands, Workflow 

In [14]:
# Start 
groupchat_result = user_proxy.initiate_chat(
    manager,
    message=task_2
)

user_proxy (to chat_manager):

Creat blog for bioinformatics based on information from clinvar

--------------------------------------------------------------------------------

Next speaker: planner


>>>>>>>> USING AUTO REPLY...
planner (to chat_manager):

Creating a blog that focuses on bioinformatics using information sourced from ClinVar involves several steps, each requiring specific expertise and actions. Below is a detailed breakdown of the task, along with instructions tailored for team members such as web scrapers, engineers, content creators, and others.

### Step 1: Define the Blog’s Scope and Objectives
- **Task for Content Strategists**:
  - Define the target audience (e.g., bioinformatics professionals, researchers, medical personnel).
  - Specify the type of content (e.g., research updates, tutorials, case studies, ethical discussions).
  - Determine the blog’s frequency and length of posts.

### Step 2: Data Collection 
ClinVar is a freely accessible, public archive of

In [17]:
# Start 
groupchat_result = user_proxy.initiate_chat(
    manager,
    message=task_3
)

user_proxy (to chat_manager):

Create summarization based on the resource given, and make it structured and give the conclusion

--------------------------------------------------------------------------------

Next speaker: data_sources_reader


>>>>>>>> USING AUTO REPLY...
data_sources_reader (to chat_manager):

To help with summarizing your resource, could you please provide the text or data that you would like to be summarized? Once I have that information, I can create a structured summary and provide a conclusion based on the content.

--------------------------------------------------------------------------------

Next speaker: data_sources_reader


>>>>>>>> USING AUTO REPLY...
data_sources_reader (to chat_manager):

Sure, to provide a structured summary and conclusion, I'll need the resource or information you want summarized. Please paste the content here or describe the data source in detail so I can assist you effectively.

--------------------------------------------------